In [ ]:
#!python --version

In [ ]:
#from __future__ import print_function
import numpy as np 
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [ ]:
# !pip uninstall pandas -y

In [ ]:
# !pip install pandas==0.25.1

In [ ]:
import pandas as pd
pd.__version__

'0.25.1'

In [ ]:
# !pip uninstall scikit-learn -y

In [ ]:
# !pip install scikit-learn==0.21.2

In [ ]:
import sklearn
sklearn.__version__

'0.21.2'

In [ ]:
import pickle

In [ ]:
import datetime 

def dateTimeFunction(date, time):
  month,day,year = dateFunc(date)
  hour,minute,second = timeFunc(time)
  dt = datetime.datetime(int(year),int(month),int(day),int(hour),int(minute),int(second))
  return dt

def dateFunc(date):
  if '/' in date:
    m,d,y = date.split('/')
    return m,d,y
  elif '-' in date:
    d = date.split(' ')
    y,m,d = d[0].split('-')
    return m,d,y
  else:
    #print('malfunction')
    return None 

def timeFunc(time):
  h,m,s = time.split(':')
  return h,m,s

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
# BWZ Carb Input (grams) == 24

In [ ]:

def part1_meal_noMeal(df):
  meal_time = []
  no_meal_time = []

  foundMeal = False

  mealCounter = 0
  noMealCounter = 0

  tempMealTime = 0
  tempNoMealTime = 0

  for index,row in df.iterrows():

    
    if isNaN(row[24]) == False and float(row[24]) > 0.0 and foundMeal == False:
      foundMeal = True
      tempMealTime = dateTimeFunction(row[1], row[2])
      noMealCounter = 0
      
    elif isNaN(row[24]) == False and float(row[24]) > 0.0 and foundMeal == True: # rule 2
      tempMealTime = dateTimeFunction(row[1], row[2]) 
      mealCounter = 0
    

    if foundMeal == True:
      mealCounter += 1
    elif foundMeal == False:
      if noMealCounter == 0:
        tempNoMealTime = dateTimeFunction(row[1], row[2])
      noMealCounter += 1


    if noMealCounter == 24:
      no_meal_time.append(tempNoMealTime)
      tempNoMealTime = 0
      noMealCounter = 0

    if mealCounter == 24 and isNaN(row[24]) == False and float(row[24]) > 0.0: # rule 3
      tempMealTime = dateTimeFunction(row[1], row[2])
      meal_time.append(tempMealTime)
      tempMealTime = 0
      foundMeal = False
      mealCounter = 0
    elif mealCounter == 24: # rule 1
      meal_time.append(tempMealTime)
      tempMealTime = 0
      foundMeal = False
      mealCounter = 0

  return meal_time, no_meal_time


In [ ]:

def part1_extract_p_q(df, meal, noMeal):

  # ensure first time for meal and no meal is not before start of cgm data
  elimCount_meal = 0
  elimCount_noMeal = 0
  for i,r in df.iterrows():
    dt = dateTimeFunction(r[1],r[2])

    mdt = meal[elimCount_meal] 
    nmdt = noMeal[elimCount_noMeal]

    if mdt < dt:
      #print('SHOULD 1/2')
      elimCount_meal += 1
    
    if nmdt < dt:
      #print('SHOULD 2/2')
      elimCount_noMeal += 1

    if mdt > dt and nmdt > dt:
      break
  
  meal = meal[elimCount_meal:]
  noMeal = noMeal[elimCount_noMeal:]

  p = []
  q = []

  mealIndex = 0
  noMealIndex = 0

  indexCounter = 0

  mealDone = False
  noMealDone = False

  for index, row in df.iterrows():

    mealDateTime = meal[mealIndex]
    noMealDateTime = noMeal[noMealIndex]

    dateTime = dateTimeFunction(row[1], row[2])

    if mealDateTime < noMealDateTime and mealDone == False: 
      if indexCounter > len(df) - 24:
        mealDone = True
        #print("Big Bad 1a index counter is " + str(indexCounter))
      elif dateTime > mealDateTime and mealIndex < len(meal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter-6:indexCounter+24]
        # if len(arr) != 30:
        #   print("Big Bad 2b index counter is " + str(indexCounter))
        # else:
        p.append(arr)
        mealIndex += 1
      elif dateTime > mealDateTime and mealIndex == len(meal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter-6:indexCounter+24]
        p.append(arr)
        mealDone = True


    elif mealDateTime > noMealDateTime and noMealDone == False:
      if indexCounter > len(df) - 24:
        noMealDone = True
        #print("Big Bad 3c index counter is " + str(indexCounter))
      elif dateTime > noMealDateTime and noMealIndex < len(noMeal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter:indexCounter+24]
        # if len(arr) != 24:
        #   print("Big Bad 4d")
        # else:
        q.append(arr)
        noMealIndex += 1
      elif dateTime > noMealDateTime and noMealIndex == len(noMeal) - 1:
        arr = df['Sensor Glucose (mg/dL)'][indexCounter:indexCounter+24]
        q.append(arr)
        noMealDone = True      

    indexCounter += 1
  return p, q

In [ ]:
def handle_missing_data(p,q):
  new_p = []
  new_q = []
  for i in p:
    drop = False
    for j in i:
      if isNaN(j):
        drop = True
    if drop == False:
      new_p.append(i)
  
  for i in q:
    drop = False
    for j in i:
      if isNaN(j):
        drop = True
    if drop == False:
      new_q.append(i)

  return new_p, new_q

In [ ]:

# Part 2 : Feature Extraction

# feature 1
def feature0(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff 

def feature00(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = 0 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff

def feature000(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = 6 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  return tdiff 

# feature 1
def feature1(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min 

def featureA(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = 0 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min

def featureAZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = 6 #x.index(cgm_min)
  max_i = x.index(cgm_max)
  tdiff = max_i - min_i
  td_in_min = tdiff * 5
  return td_in_min 

# feature 2
def feature2(x):
  cgm_min = min(x)
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 

def featureB(x):
  cgm_min = x[0]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 

def featureBZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  cgN = cg / cgm_min
  return cgN 
  
# feature 3,4,5,6: sinusoid frequency response ~ fast fourier transform : Pf1 , f1, Pf2, f2
def feature_sfr_fft(x):
  retval = []
  fft = np.fft.fft(np.array(x))
  for i in fft[:24]:
    real = i.real
    imag = i.imag 
    retval.append(real)
    retval.append(imag) 
  return retval

# feature 7: slope of cgm diff over time from meal start to cgm max
def feature7(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope 

def featureC(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope

def featureCZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope 

def feature70(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope 

def featureC0(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope

def featureCZ0(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return 0
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope 

def feature80(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt 
  slope = cg / dt 
  return slope / dt

def featureD0(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt * 5
  slope = cg / dt 
  return slope / dt

def featureDZ0(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  # dt_in_min = dt 
  slope = cg / dt 
  return slope / dt


def feature8(x):
  cgm_min = min(x)
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min

def featureD(x):
  cgm_min = x[0]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min

def featureDZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  min_i = x.index(cgm_min)
  max_i = x.index(cgm_max)
  cg = cgm_max - cgm_min
  if cg == 0:
    return 0
  dt = max_i - min_i 
  if dt == 0:
    return
  dt_in_min = dt * 5
  slope = cg / dt_in_min 
  return slope / dt_in_min


def feature9(x):
  cgm_min = min(x)
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg

def featureE(x):
  cgm_min = x[0]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg

def featureEZ(x):
  cgm_min = x[6]
  cgm_max = max(x)
  cg = cgm_max - cgm_min
  return cg


def feature10(x):
  return max(x)

def feature11(x):
  return min(x)



In [ ]:
def part2_feature_extraction(npq):
  fpq = []

  for i in npq: 
    f = []

    f1 = feature1(i)
    f.append(f1)

    f2 = feature2(i)
    f.append(f2)

    # f3 = feature_sfr_fft(i)
    # for fi in f3:
    #   f.append(fi)

    f7 = feature7(i)
    f.append(f7)

    f8 = feature8(i)
    f.append(f8)

    f9 = feature9(i) 
    f.append(f9) 

    # f10 = feature10(i)
    # f.append(f10)

    # f11 = feature11(i)
    # f.append(f11)

    fA = featureA(i)
    f.append(fA)

    fB = featureB(i)
    f.append(fB)

    fC = featureC(i)
    f.append(fC)

    fD = featureD(i)
    f.append(fD) 

    fE = featureE(i)
    f.append(fE)

    fAZ = featureAZ(i)
    f.append(fAZ)

    fBZ = featureBZ(i)
    f.append(fBZ)

    fCZ = featureCZ(i)
    f.append(fCZ)

    fDZ = featureDZ(i)
    f.append(fDZ) 

    fEZ = featureEZ(i)
    f.append(fEZ)

    f0 = feature0(i)
    f.append(f0)
    
    f00 = feature00(i)
    f.append(f00)

    f000 = feature000(i)
    f.append(f000)

    # f70 = feature70(i)
    # f.append(f70)

    # fC0 = featureC0(i)
    # f.append(fC0)

    # fCZ0 = featureCZ0(i)
    # f.append(fCZ0)

    # f80 = feature80(i)
    # f.append(f80)

    # fD0 = featureD0(i)
    # f.append(fD0)

    # fDZ0 = featureDZ0(i)
    # f.append(fDZ0)


    fpq.append(f)
  
  return fpq

In [ ]:
#def main():

# read in data
df_insulin = pd.read_csv('InsulinData.csv', low_memory=False)
df_cgm = pd.read_csv('CGMData.csv', low_memory=False)
#
dfi2 = pd.read_csv('Insulin_patient2.csv', low_memory=False)
dfc2 = pd.read_csv('CGM_patient2.csv', low_memory=False)

In [ ]:
# reverse order of dataframe so time is in chronological order
df_insulin = df_insulin.iloc[::-1]
df_cgm = df_cgm.iloc[::-1]
#
dfi2 = dfi2.iloc[::-1]
dfc2 = dfc2.iloc[::-1]

In [ ]:
# Part 1 
meal, noMeal = part1_meal_noMeal(df_insulin)
p, q = part1_extract_p_q(df_cgm, meal, noMeal)
#
meal2, noMeal2 = part1_meal_noMeal(dfi2.iloc[: , 1:])
p2, q2 = part1_extract_p_q(dfc2.iloc[: , 1:], meal2, noMeal2)

# handle missing data
new_p, new_q = handle_missing_data(p,q)
#
new_p2, new_q2 = handle_missing_data(p2,q2)

In [ ]:
# Part 2
np_ = [list(i) for i in new_p]
nq_ = [list(i) for i in new_q]
#
np2 = [list(i) for i in new_p2]
nq2 = [list(i) for i in new_q2]

fp  = part2_feature_extraction(np_)
fq = part2_feature_extraction(nq_)
fp2 = part2_feature_extraction(np2)
fq2 = part2_feature_extraction(nq2)

In [ ]:
# Part 3
# make matrix with label vector
meal_features = fp + fp2
no_meal_features = fq + fq2 

meal_labels = [1 for i in range(len(meal_features))]
no_meal_labels = [0 for i in range(len(no_meal_features))]

features = meal_features + no_meal_features
labels = meal_labels + no_meal_labels

matrix = []
for i in range(len(features)):
  matrix.append(features[i] + [labels[i]])

np.random.shuffle(matrix) 

train_data = [] 
test_data = []  
for i in range(len(matrix)):
  if i / len(matrix) < 0.8:
    train_data.append(matrix[i])
  else:
    test_data.append(matrix[i])



In [ ]:
train_features = [i[:-1] for i in train_data]
train_labels = [i[-1] for i in train_data]

test_features = [i[:-1] for i in test_data]
test_labels = [i[-1] for i in test_data]

In [ ]:
# train with svm or decision tree
# Use k fold cross validation on the training data to evaluate your recognition system: 80% training, 20% validation
# precision, recall, f1 score to validate

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(train_features, train_labels)
scores = cross_val_score(clf, test_features, test_labels, cv=5)
# 

/usr/local/lib/python3.7/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:442: 

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.65 accuracy with a standard deviation of 0.00


In [ ]:
clf.predict(test_features)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
print(test_labels)

[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 

In [ ]:
# from sklearn import tree

clf2 = tree.DecisionTreeClassifier()
clf2 = clf2.fit(train_features, train_labels)

scores2 = cross_val_score(clf2, test_features, test_labels, cv=5)


/usr/local/lib/python3.7/dist-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores2.mean(), scores2.std()))

0.86 accuracy with a standard deviation of 0.03


In [ ]:
clf2.predict(test_features)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,

In [ ]:
# store it in pickle
filename = 'finalized_model.sav'
pickle.dump(clf2, open(filename, 'wb'))


In [ ]:
#	Write a function with
# input of N x 24 cgm values matrix and
# output of N x 1 matrix into a results.csv 
# 1 if meal, 0 if no meal